In [1]:
# import gradio as gr
from transformers import BertTokenizer,T5Tokenizer, T5ForConditionalGeneration, MT5ForConditionalGeneration, Text2TextGenerationPipeline
import sys, os, torch
import json
sys.path.append(os.getcwd())
import gradio as gr

In [2]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
tokenizer = BertTokenizer.from_pretrained('./models/my_t5_base/')
model = MT5ForConditionalGeneration.from_pretrained('../v1.3/').to(device)

In [3]:
demo = gr.Blocks()


def inference(text,tags):
    prefixed_prompts = ''
    for tag in tags:
        prefixed_prompts = prefixed_prompts+f'<实体><{tag}>'
        
    print(tags)
     
    prefixed_prompts = prefixed_prompts+'<文本>'
    print(prefixed_prompts)
    
    input_sequences = [prefixed_prompts+text]
    input_sequences = [text.lower() for text in input_sequences]


    encoding = tokenizer(input_sequences,padding="longest", max_length=512, \
                         truncation=True, return_tensors="pt",)
    input_ids, attention_mask = encoding.input_ids.to(device), encoding.attention_mask.to(device)

    # inference
    outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask,\
                         do_sample=False,eos_token_id=tokenizer.sep_token_id,num_beams = 10, max_length= 512,
                        decoder_start_token_id=tokenizer.cls_token_id)
    
    preds = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    preds = ''.join(preds.split(' '))
    preds = preds.split(',')
    preds = [item.replace(')','').replace('(','').split(':') for item in preds]
    preds = [item for item in preds if item[0] in tags]
    print(preds)
    
    return preds

## format prediction results from list of tuples -> dictionary with offsets
def add_offset(text, preds):
   
    lower_text = text.lower()
    ner_dict = {}
    results = []
    for pred in preds:

        start = lower_text.find(pred[1])
        if start==-1:
            pass

        else:
            tag = pred[0]
            if tag not in ner_dict:
                end = start+len(pred[1])
                tag_text = text[start:end]
                tag = pred[0]
                ner_dict[tag] = {tag_text:[[start,end-1]]}
            else:
                end = start+len(pred[1])
                tag_text = text[start:end]

                if tag_text in ner_dict[tag]:
                    old_end = ner_dict[tag][tag_text][-1][1]
        #                     print(idx)
                    new_start = lower_text[old_end:].find(pred[1])
                    new_start = new_start+old_end
                    new_end = new_start+len(pred[1])
                    ner_dict[tag][tag_text].append([new_start,new_end-1])
                else:
                    ner_dict[tag].update({tag_text:[[start,end-1]]})
    print(ner_dict)
    return ner_dict

## format prediction results from dictionary with offsets -> gradio highlighted text output [(word, tag)...(word,tag)]
def dict_to_gradio_output(text, ner_dict):
    
    results = []
    for tag,value in ner_dict.items():
        for word, offsets in value.items():
            for offset in offsets:
                results.append([offset[0],offset[1],word,tag])   
                
    print(results)
    results = sorted(results, key=lambda tup: tup[0])
    print(results)
            
    newres = []
    ntext = len(text)+1
    for line in results:
        if len(newres)==0:
            if line[0]==0:
                newres.append(line)
            else:
                begin = 0
                end = line[0]
                newres.append([begin,end-1,text[begin:end],None])
                newres.append(line)
        else:
            if line[0]-newres[-1][1] > 1:
                begin = newres[-1][1]+1
                end = line[0]
                newres.append([begin,end, text[begin:end], None])
            newres.append(line)
    if len(newres)==0:
        return [(text,None)]
    
    if newres[-1][1] < ntext-1:
        begin = newres[-1][1] + 1
        end = ntext-1
        newres.append([begin, end, text[begin:end], None])
                
    
    gradio_result = []
    for line in newres:
        gradio_result.append((line[2],line[3]))
            
    print(gradio_result)
    return gradio_result
    
    

def entity_extraction(text,tags):

    predictions = inference(text,tags)
    ner_dict = add_offset(text,predictions)
    final_output = dict_to_gradio_output(text,ner_dict)
    
    return final_output

demo = gr.Blocks()

# inp = gr.Textbox(placeholder="请输入文本", label="输入文本")
# inp1 = gr.Textbox(placeholder="请输入文本", label="输入文本")


cluener_tags = ['名称','公司','游戏','组织','电影', '地点','职位','政府','景点','书籍']
boson_tags = ['产品','时间', '名称', '组织', '地点','公司'] 
pd_tags = ['地点','名称','组织','时间']
msra_tags = ['地点','名称','组织']
ontonotes_tags = ['地缘政治实体','地点','名称','组织']
chinese_address_tags = ['省份', '城市', '区', '街道', '社区','兴趣点', '路', '路号', '次兴趣点', '产业园', '楼号', '路口', '方位', '单元','楼层', '距离', '村组']
resume_tags = ['名称','国籍', '民族', '职位', '学历', '公司', '专业', '籍贯']
ecommerce_tags = ['品牌','商品']


tags = ['品牌','商品', '地点','名称','组织','地缘政治实体','国籍', '民族', '职位', '学历', '公司', '专业', '籍贯','游戏','电影', '政府','景点','书籍',
'产品','时间', '省份', '城市', '区', '街道', '社区','兴趣点', '路', '路号', '次兴趣点', '产业园', '楼号', '路口', '方位', '单元','楼层', '距离', '村组']

extraction_iface = gr.Interface(
    fn=entity_extraction,
    inputs=['text',gr.CheckboxGroup(tags),],
    outputs=gr.outputs.HighlightedText(label="命名实体识别"),
    examples=[['九城宣布将于美国东部时间2012年3月21日美国股市收盘后(北京时间3月22日)发布2011财年Q4及全年未审计财务报告',boson_tags],
              ["陕西省西安市碑林区东关南街街道仁厚南路高山流水星币传说0号楼",chinese_address_tags],
              ['百雀羚洗面奶 男士深层净化洁面泡沫150m 控油补水 收敛毛孔 官方',ecommerce_tags],
              ["怎样做才能避免拒签？新浪出国频道邀请到美国使馆签证处的签证官江德力（charles",cluener_tags],
              ["致公党中央领导人多次参加中共中央和国务院举行的民主党派人士座谈会、协商会，参与国家大政方针的协商，认真履行参政议政、民主监督职能；",msra_tags],
              ["1928年3月,土肥原出任张作霖的顾问,后一手策划了“皇姑屯事件”。1931年在天津设立特务机关,土肥原任机关长,后将溥仪从天津诱至大连,拼凑伪满傀儡政权。之后,土肥原被调往哈尔滨出任特务机关长,镇压东北抗日武装力量。",pd_tags],
              ["1955年出生，兰州大学经济系硕士毕业，浙江大学经济学教授、博士生导师。",resume_tags],
              ["报警人称其丈夫（小明，18112345678，47周岁，12345678123456781X，有精神病（医院确诊过）五分钟前醉酒驾车从金桥路杨高南路出去，车辆信息：浙AH057Y，银灰色起亚轿车，请各单位注意发现，谢谢配合！",['名称','地点','公司']],
              ["黑色衬衣，蓝色牛仔裤的男子，2022年5月31日早上9点半左右，深圳湾科技园12A",['时间','地点']],
                ]
)

extraction_iface.launch(server_name="0.0.0.0")

# launcher = gr.Parallel(
#             extraction_iface,
#             examples=[
#                 "黑色衬衣，蓝色牛仔裤的男子，2022年5月31日早上9点半左右，深圳湾科技园12A",
#                 "穿白色上衣的美团骑手 戴墨镜",
#                 "补充 车牌，沪HN7892 蓝色出租车，沿着往长宁路往西逃逸 民警处罚违章车辆时被一辆疑似海博 蓝色出租车，尾号7862 带倒拖行，现民警受伤",
#                 "报警人称其丈夫（小明，18112345678，47周岁，12345678123456781X，有精神病（医院确诊过）五分钟前醉酒驾车从金桥路杨高南路出去，车辆信息：浙AH057Y，银灰色起亚轿车，请各单位注意发现，谢谢配合！",
#                 "一男子买了菜刀上了 沪GU2976 强生出租车 黑灰色衣服， 1米8左右，40岁左右。请你局工作中注意发现。"
#                 ]
# )

Running on local URL:  http://localhost:7860/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x7f18b769ed00>, 'http://localhost:7860/', None)

In [ ]:
'省份', '城市', '区', '街道', '社区','兴趣点', '路', '路号', '次兴趣点', '产业园', '楼号', '路口', '方位', '单元','楼层', '距离', '村组'

In [ ]:
a = ['品牌','商品' '地点','名称','组织','地缘政治实体','国籍', '民族', '职位', '学历', '公司', '专业', '籍贯', '公司','游戏','电影', '政府','景点','书籍',
'产品','时间', '省份', '城市', '区', '街道', '社区','兴趣点', '路', '路号', '次兴趣点', '产业园', '楼号', '路口', '方位', '单元','楼层', '距离', '村组']

In [ ]:
data = [[1,2,3], [4,5,6], [7,8,9]]
sorted_by_second = sorted(data, key=lambda tup: tup[1],reverse=True)

In [ ]:
data

In [ ]:
sorted_by_second